In [55]:
import numpy as np
import osr_methods as osr
import line_search_methods as lsm
import ocr_methods as ocr
import numdifftools as nd
import matplotlib.pyplot as plt
from timeit import default_timer as timer

### Definição do Ponto Inicial

In [56]:
#exterior
x = np.array([-1, 5])

#interior
#x = np.array([2., -2.])

### Escolha dos métodos de OSR e OCR

In [57]:
# Metodos OCR
# 1 - Penalidade
# 2 - Barreira
metodo_ocr = 1

# Metodos OSR
# 1 - Univariante
# 2 - Powell
# 3 - Stepest Descent
# 4 - Newton-Raphson
# 5 - Fletcher-Reeves
# 6 - BFGS
metodo_osr = 6

### Controle Numérico

In [58]:
# numero maximo de iteracoes na OSR
maxiter = 200

# tolerancia para convergencia do gradiente na OSR
tol_conv = 1E-5

# tolerancia para a busca unidirecional na OSR
tol_search = 1E-6

# delta alpha do passo constante na OSR
line_step = 1E-2

#epsilon da maquina
eps = 1E-8

#parametros ocr
if metodo_ocr == 1:
    #penalidade
    r = 1
    beta = 10
elif metodo_ocr == 2:
    #barreira
    r = 1
    beta = 0.1

#tolerancia OCR
tol = 1E-3

ctrl_num_osr = [maxiter, tol_conv, tol_search, line_step, eps]

### Definição de f

In [59]:
def f(x):
    return (x[0]-1)**2 + (x[1]-2)**2

def grad_f(x):
    return np.array([2*(x[0]-1), 2*(x[1]-2)])

def hess_f(x):
    hess = np.zeros((2,2), dtype=float)
    hess[0,:] = np.array([2.,0.])
    hess[1,:] = np.array([0.,2.])
    return hess

### Definição das restrições de igualdade

In [60]:
def hk1(x):
    return x[0] + x[1] - 2

def grad_hk1(x):
    return np.array([1.,1.])

def hess_hk1(x):
    return np.zeros((2,2), dtype=float)

### Definição das restrições de desigualdade

In [61]:
def cl1(x):
    return -x[0]**2 + x[1]

def grad_cl1(x):
    return np.array([-2.0*x[0], 1.])

def hess_cl1(x):
    hess = np.zeros((2,2), dtype=float)
    hess[0,:] = np.array([-2., 0.])
    hess[1,:] = np.array([0., 0.])
    return hess



### Listas de Restrições e suas derivadas

In [62]:
hk_list = [hk1]
grad_hk_list = [grad_hk1]
hess_hk_list = [hess_hk1]

cl_list = [cl1]
grad_cl_list = [grad_cl1]
hess_cl_list = [hess_cl1]

#para o metodo de penalidade
#controle de quais cls irao montar a phi
cl_mont = []
if metodo_ocr == 1:
    for cl in cl_list:
        if cl(x) > 0:
            cl_mont.append(1)
        else:
            cl_mont.append(0)
            
params = [f, grad_f, hess_f, hk_list, grad_hk_list, hess_hk_list, cl_list, grad_cl_list, hess_cl_list, cl_mont]


In [63]:

if metodo_ocr == 1:
    parc = (1/2)*r*ocr.p_penal(x, params)
elif metodo_ocr == 2:
    parc = r*ocr.b_bar(x, params)
    
listP_OCR = []
listP_OCR.append(x)

passos_OCR = 0
while(parc > tol):
    passos_OCR = passos_OCR + 1
    if passos_OCR > 1:
        r = beta*r
        if metodo_ocr == 1:
            cl_mont = []
            for cl in cl_list:
                if cl(x) > 0:
                    cl_mont.append(1)
                else:
                    cl_mont.append(0)
    listP_OSR, passos_OSR, flag_conv_OSR, tempoExec_OSR = osr.osr_ctrl(x, params, r, ctrl_num_osr, metodo_ocr, metodo_osr)
    x = listP_OSR[-1]
    listP_OCR.append(x)
    if metodo_ocr == 1:
        parc = (1/2)*r*ocr.p_penal(x, params)
    elif metodo_ocr == 2:
        parc = r*ocr.b_bar(x, params)
    print(f'{passos_OCR}, iteracoes={passos_OSR} x={x}, r={r}')
    

1, iteracoes=7 x=[1.08085022 1.5218467 ], r=1
2, iteracoes=4 x=[1.01953106 1.09996024], r=10
3, iteracoes=200 x=[1.00219299 1.01098901], r=100
4, iteracoes=200 x=[1.00022192 1.00110956], r=1000
5, iteracoes=200 x=[1.00002225 1.00011107], r=10000
